Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
# Constants
BATCH_SIZE = 16
PATCH_SIZE = 5
CONV_DEPTH = 16
NUM_HIDDEN = 64
NUM_STEPS = 1001
LEARNING_RATE = 0.05

# Print the configuration
print("Batch size:",BATCH_SIZE)
print("Patch size:",PATCH_SIZE)
print("Convolution Depth:",CONV_DEPTH)
print("Steps:",NUM_STEPS)
print("Learning Rate:",LEARNING_RATE)
#print("Regularization Constant:", REG_CONSTANT)
#print("Dropout probability:", KEEP_PROB)

# Initialize the graph
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, num_channels, CONV_DEPTH], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([CONV_DEPTH]))
    layer2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, CONV_DEPTH, CONV_DEPTH], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[CONV_DEPTH]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * CONV_DEPTH, NUM_HIDDEN], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
    layer4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
# Print the table format
print('{:^20}'.format('Minibatch'),end="")
print('{:^20}'.format('Time'),'{:^20}'.format('Loss'),end="")
print('{:^20}'.format('Train accuracy'),'{:^20}'.format('Validation accuracy'))

# Initialize the averages
batch_avg_loss = 0.0
batch_avg_acc = 0.0
batch_step = 0
start_time = time.clock()

# Run the session
with tf.Session(graph=graph) as session:
    # Initialize all the variables
    tf.initialize_all_variables().run()
    
    # Loop through the steps
    for step in range(NUM_STEPS):
    
        # Calculate the offset
        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        
        # Form a mini batch
        batch_data = train_dataset[offset:(offset + BATCH_SIZE), :, :, :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
        
        # Associate the appropriate data
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the learning for this batch
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
       # Calculate the averages
        if step != 0:
            batch_avg_loss = ((batch_avg_loss * (step - batch_step - 1)) + l) \
                                / (step - batch_step)
            batch_avg_acc = ((batch_avg_acc * (step - batch_step - 1)) + accuracy(predictions, batch_labels)) \
                                / (step - batch_step)
                
        # At periodic intervals, print out the statistics
        if ((step % (int)(NUM_STEPS/15) == 1) or (step == NUM_STEPS -1)): 
            # Print the values
            print('{:^20}'.format(str(step)),end="")
            print('{:^20}'.format(str(time.clock() - start_time)),end="")
            print('{:^20}'.format(str(batch_avg_loss)),end="")
            print('{:^20}'.format(str(batch_avg_acc)),end="")
            print('{:^20}'.format(str(accuracy(valid_prediction.eval(), valid_labels))))

            # Reset the values
            start_time = time.clock()
            batch_avg_loss = 0.0
            batch_avg_acc = 0.0
            batch_step = step
            
    # Print the test accuracy
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [ ]:
# Constants
BATCH_SIZE = 16
PATCH_SIZE = 5
CONV_DEPTH = 16
NUM_HIDDEN = 64
NUM_STEPS = 1001
LEARNING_RATE = 0.05
POOL_SIZE = 2
POOL_STRIDE = 2

# Print the configuration
print("Batch size:",BATCH_SIZE)
print("Patch size:",PATCH_SIZE)
print("Convolution Depth:",CONV_DEPTH)
print("Steps:",NUM_STEPS)
print("Learning Rate:",LEARNING_RATE)
#print("Regularization Constant:", REG_CONSTANT)
#print("Dropout probability:", KEEP_PROB)

# Initialize the graph
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, num_channels, CONV_DEPTH], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([CONV_DEPTH]))
    layer2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, CONV_DEPTH, CONV_DEPTH], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[CONV_DEPTH]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 2 * image_size // 2 * CONV_DEPTH, NUM_HIDDEN], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
    layer4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.max_pool(conv1, ksize=[1, POOL_STRIDE, POOL_STRIDE, 1], 
                              strides=[1, POOL_STRIDE, POOL_STRIDE, 1], padding='SAME')
        conv1 = tf.nn.relu(conv1 + layer1_biases)
        #conv2 = tf.nn.conv2d(conv1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        #conv2 = tf.nn.max_pool(conv2, ksize=[1, POOL_STRIDE, POOL_STRIDE, 1], 
        #                      strides=[1, POOL_STRIDE, POOL_STRIDE, 1], padding='SAME')
        #conv2 = tf.nn.relu(conv2 + layer2_biases)
        shape = conv1.get_shape().as_list()
        output = tf.reshape(conv1, [shape[0], shape[1] * shape[2] * shape[3]])
        output = tf.nn.relu(tf.matmul(output, layer3_weights) + layer3_biases)
        return tf.matmul(output, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
# Print the table format
print('{:^20}'.format('Minibatch'),end="")
print('{:^20}'.format('Time'),'{:^20}'.format('Loss'),end="")
print('{:^20}'.format('Train accuracy'),'{:^20}'.format('Validation accuracy'))

# Initialize the averages
batch_avg_loss = 0.0
batch_avg_acc = 0.0
batch_step = 0
start_time = time.clock()

# Run the session
with tf.Session(graph=graph) as session:
    # Initialize all the variables
    tf.initialize_all_variables().run()
    
    # Loop through the steps
    for step in range(NUM_STEPS):
    
        # Calculate the offset
        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        
        # Form a mini batch
        batch_data = train_dataset[offset:(offset + BATCH_SIZE), :, :, :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
        
        # Associate the appropriate data
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the learning for this batch
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
       # Calculate the averages
        if step != 0:
            batch_avg_loss = ((batch_avg_loss * (step - batch_step - 1)) + l) \
                                / (step - batch_step)
            batch_avg_acc = ((batch_avg_acc * (step - batch_step - 1)) + accuracy(predictions, batch_labels)) \
                                / (step - batch_step)
                
        # At periodic intervals, print out the statistics
        if ((step % (int)(NUM_STEPS/15) == 1) or (step == NUM_STEPS -1)): 
            # Print the values
            print('{:^20}'.format(str(step)),end="")
            print('{:^20}'.format(str(time.clock() - start_time)),end="")
            print('{:^20}'.format(str(batch_avg_loss)),end="")
            print('{:^20}'.format(str(batch_avg_acc)),end="")
            print('{:^20}'.format(str(accuracy(valid_prediction.eval(), valid_labels))))

            # Reset the values
            start_time = time.clock()
            batch_avg_loss = 0.0
            batch_avg_acc = 0.0
            batch_step = step
            
    # Print the test accuracy
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
# Execution constants
BATCH_SIZE = 16
NUM_STEPS = 6001

# Convolution constants
PATCH_SIZE = 5
CONV1_DEPTH = 16
CONV2_DEPTH = 32
POOL1_STRIDE = 2
POOL2_STRIDE = 3

# NN constants
NUM_HIDDEN1 = 512
NUM_HIDDEN2 = 128
NUM_HIDDEN3 = 32
LEARNING_RATE = 0.1
LR_DECAY_STEPS = (NUM_STEPS/1)
LR_DECAY_RATE = 0.9
KEEP_PROB = 0.9
REG_CONSTANT = 1e-5

# Print the configuration
print("Batch size:",BATCH_SIZE)
print("Patch size:",PATCH_SIZE)
print("Convolution Depth:",CONV1_DEPTH, CONV2_DEPTH)
print("Steps:",NUM_STEPS)
print("Learning Rate:",LEARNING_RATE)
#print("Regularization Constant:", REG_CONSTANT)
#print("Dropout probability:", KEEP_PROB)

# Initialize the graph
graph = tf.Graph()
with graph.as_default():
    # Input data.
    with tf.name_scope('input'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    # Variables.
    l1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, num_channels, CONV1_DEPTH], stddev=0.1))
    l1_biases = tf.Variable(tf.zeros([CONV1_DEPTH]))
    
    l2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, CONV1_DEPTH, CONV2_DEPTH], stddev=0.1))
    l2_biases = tf.Variable(tf.zeros([CONV2_DEPTH]))
    
    l3_weights = tf.Variable(tf.truncated_normal([image_size // 7 * image_size // 7 * CONV2_DEPTH, NUM_HIDDEN1], stddev=0.1))
    l3_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN1]))
    l4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN1, NUM_HIDDEN2], stddev=0.1))
    l4_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN2]))
    l5_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN2, NUM_HIDDEN3], stddev=0.1))
    l5_biases = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN3]))
    l6_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN3, num_labels], stddev=0.1))
    l6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # Layer 1 convolution with stride 1
        l1_conv = tf.nn.conv2d(data, l1_weights, [1, 1, 1, 1], padding='SAME')
        l1_conv = tf.nn.dropout(tf.nn.relu(l1_conv + l1_biases), KEEP_PROB)
        
        # Layer 1 pooling with stride 2
        l1_pool = tf.nn.max_pool(l1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        # Layer 1 normalization
        l1_norm = tf.nn.lrn(l1_pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

        # Layer 2 convolution with stride 2
        l2_conv = tf.nn.conv2d(l1_pool, l2_weights, [1, 1, 1, 1], padding='SAME')
        l2_conv = tf.nn.dropout(tf.nn.relu(l2_conv + l2_biases), KEEP_PROB)
        
        # Layer 2 pooling with stride 2
        l2_pool = tf.nn.max_pool(l2_conv, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')
        
        # Layer 1 normalization
        l2_norm = tf.nn.lrn(l2_pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
        
        # Reshape for a NN
        shape = l2_pool.get_shape().as_list()
        l3_input = tf.reshape(l2_pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # Layer 3 - Hidden layer NN 1
        l3_nn = tf.nn.dropout(tf.nn.relu(tf.matmul(l3_input, l3_weights) + l3_biases), KEEP_PROB)

        # Layer 4 - Hidden Layer NN 2
        l4_nn = tf.nn.dropout(tf.nn.relu(tf.matmul(l3_nn, l4_weights) + l4_biases), KEEP_PROB)

        # Layer 4 - Hidden Layer NN 2
        l5_nn = tf.nn.dropout(tf.nn.relu(tf.matmul(l4_nn, l5_weights) + l5_biases), KEEP_PROB)
        
        # Output layer
        output = tf.matmul(l5_nn, l6_weights) + l6_biases
        
        return output
        
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # L2 regularization
    regularizer = REG_CONSTANT * (tf.nn.l2_loss(l1_weights) + tf.nn.l2_loss(l1_biases) +\
                                  tf.nn.l2_loss(l2_weights) + tf.nn.l2_loss(l2_biases) +\
                                 tf.nn.l2_loss(l3_weights) + tf.nn.l2_loss(l3_biases) +\
                                 tf.nn.l2_loss(l4_weights) + tf.nn.l2_loss(l4_biases) +\
                                 tf.nn.l2_loss(l5_weights) + tf.nn.l2_loss(l5_biases) +\
                                 tf.nn.l2_loss(l6_weights) + tf.nn.l2_loss(l6_biases))
    
    # Add the regularization to the loss
    loss = tf.reduce_mean(loss + regularizer)
    
    # Optimizer with a decaying learning rate
    learning_rate = tf.train.exponential_decay(LEARNING_RATE, global_step, LR_DECAY_STEPS, LR_DECAY_RATE, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
# Print the table format
print('{:^20}'.format('Minibatch'),end="")
print('{:^20}'.format('Time'),'{:^20}'.format('Loss'),end="")
print('{:^20}'.format('Train accuracy'),'{:^20}'.format('Validation accuracy'))

# Initialize the averages
batch_avg_loss = 0.0
batch_avg_acc = 0.0
batch_step = 0
start_time = time.clock()

writer = tf.train.SummaryWriter("conv.log", graph=tf.get_default_graph())

# Run the session
with tf.Session(graph=graph) as session:
    # Initialize all the variables
    tf.initialize_all_variables().run()
    
    # Loop through the steps
    for step in range(NUM_STEPS):
    
        # Calculate the offset
        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        
        # Form a mini batch
        batch_data = train_dataset[offset:(offset + BATCH_SIZE), :, :, :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
        
        # Associate the appropriate data
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the learning for this batch
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
       # Calculate the averages
        if step != 0:
            batch_avg_loss = ((batch_avg_loss * (step - batch_step - 1)) + l) \
                                / (step - batch_step)
            batch_avg_acc = ((batch_avg_acc * (step - batch_step - 1)) + accuracy(predictions, batch_labels)) \
                                / (step - batch_step)
                
        # At periodic intervals, print out the statistics
        if ((step % (int)(NUM_STEPS/15) == 1) or (step == NUM_STEPS -1)): 
            # Print the values
            print('{:^20}'.format(str(step)),end="")
            print('{:^20}'.format(str(time.clock() - start_time)),end="")
            print('{:^20}'.format(str(batch_avg_loss)),end="")
            print('{:^20}'.format(str(batch_avg_acc)),end="")
            print('{:^20}'.format(str("0")))
            #print('{:^20}'.format(str(accuracy(valid_prediction.eval(), valid_labels))))

            # Reset the values
            start_time = time.clock()
            batch_avg_loss = 0.0
            batch_avg_acc = 0.0
            batch_step = step
            
    # Print the test accuracy
    print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Batch size: 16
Patch size: 5
Convolution Depth: 16 32
Steps: 6001
Learning Rate: 0.1
     Minibatch              Time                 Loss           Train accuracy    Validation accuracy 
         1           0.4063629999999421    3.62212729454            6.25                 0          
        401          18.39096600000005     1.62154690925         42.265625               0          
        801          18.29506099999992     0.71282111479         79.046875               0          
        1201         18.29594099999997     0.637463312978        80.984375               0          
        1601         18.760423999999944    0.55403021324          84.0625                0          
        2001         19.022718999999938    0.537980286805         84.46875               0          
        2401         18.453664000000003    0.491717620175        86.078125               0          
        2801         18.228420999999912    0.525562331993        84.953125               0          
    